In [1]:
%tensorflow_version 1.x
import tensorflow.keras as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [2]:
def preprocess_data(X, Y):
    X = K.applications.resnet50.preprocess_input(X)
    Y = K.utils.to_categorical(Y, 10)

    return X, Y

In [3]:
# Divide the data in Train and Test Datasets
(x_train, y_train),(x_test, y_test) = K.datasets.cifar10.load_data()

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

# The original size of pictures is 32 x 32, we need to upscale to 244 x 224 to 
# fit on Resnet50 
target_size = (224, 224)

model = K.applications.ResNet50(include_top=False,
                                weights='imagenet',
                                input_shape=(224, 224, 3))

170500096/170498071 [==============================] - 13s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 10s 0us/step


In [4]:
for layer in model.layers[:-32]:
    layer.trainable=False

In [6]:
model_1= K.Sequential()
model_1.add(K.layers.Lambda(lambda image: tf.image.resize(image, target_size))) 
model_1.add(model)
model_1.add(K.layers.GlobalAveragePooling2D()) 
model_1.add(K.layers.Flatten())
model_1.add(K.layers.BatchNormalization())
model_1.add(K.layers.Dense(64, activation='relu', kernel_initializer='he_normal'))
model_1.add(K.layers.Dropout(0.5))
model_1.add(K.layers.BatchNormalization())
model_1.add(K.layers.Dense(10,activation='softmax', kernel_initializer='he_normal'))

checkpoint = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
                                          monitor='val_acc',
                                          mode='max',
                                          verbose=1,
                                          save_best_only=True)

model_1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model_1.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            batch_size=32,
            epochs=20,
            verbose=1,
            callbacks=[checkpoint])

model_1.save('cifar10.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
49984/50000 [============================>.] - ETA: 0s - loss: 0.6756 - acc: 0.7892
Epoch 00001: val_acc improved from -inf to 0.84150, saving model to cifar10.h5
50000/50000 [==============================] - 367s 7ms/sample - loss: 0.6756 - acc: 0.7892 - val_loss: 0.5828 - val_acc: 0.8415
Epoch 2/20
49984/50000 [============================>.] - ETA: 0s - loss: 0.3817 - acc: 0.8815
Epoch 00002: val_acc improved from 0.84150 to 0.86500, saving model to cifar10.h5
50000/50000 [==============================] - 362s 7ms/sample - loss: 0.3818 - acc: 0.8815 - val_loss: 0.4991 - val_acc: 0.8650
Epoch 3/20
49984/50000 [============================>.] - ETA: 0s - loss: 0.2690 - acc: 0.9161
Epoch 00003: val_acc improved from 0.86500 to 0.86670, saving model to cifar10.h5
50000/50000 [==============================] - 362s 7ms/sample - loss: 0.2691 - acc: 0.9160 - val_loss: 0.5542 - val_acc: 0.8667
Epoch 4/20
49984/50000 [===========

In [7]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            multiple                  0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  8192      
_________________________________________________________________
dense_2 (Dense)              multiple                  131136    
_________________________________________________________________
dropout_1 (Dropout)          multiple                 

In [8]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________